# REGISTRIS PERIODO: 2010-09-02  a 2024-05-22.

In [1]:
import pandas as pd
import glob
import re
import numpy as np
import geopandas as gpd
from shapely.geometry import Point

In [2]:
# Ruta archivos .csv
rutas_archivos = glob.glob('/content/*.csv')  # la ruta corresponde a "Archivos" de colab.

# Crear un DataFrame vacío para concatenar todos los DataFrames
df = pd.DataFrame()

# Iterar sobre cada archivo en la lista
for ruta in rutas_archivos:
    # Cargar el archivo CSV en un DataFrame
    df_temp = pd.read_csv(ruta)
    # Agregar una columna con el nombre del archivo CSV (sin la ruta completa y sin .csv)
    nombre_archivo = ruta.split('/')[-1].replace('.csv', '')
    df_temp['Localidad'] = nombre_archivo
    # Concatenar el DataFrame temporal con el DataFrame final
    df = pd.concat([df, df_temp], ignore_index=True)

In [5]:
df.head(40)

,fecha,Estación,Precipitación,Localidad
0,2010-09-02,Durazno - Comisaría,4mm,Tala
1,2010-09-02,Gdor Echagüe - Comisaría,5mm,Tala
2,2010-09-02,Gdor Mansilla - Comisaría,5mm,Tala
3,2010-09-02,Gdor. Maciá - Comisaría,11mm,Tala
4,2010-09-02,Rosario del Tala - OSM,8mm,Tala
5,2010-09-02,Sauce Norte,7mm,Tala
6,2010-09-02,Tala - Ciudad,8mm,Tala
7,2010-09-10,Durazno - Comisaría,18mm,Tala
8,2010-09-10,Gdor Echagüe - Comisaría,24mm,Tala
9,2010-09-10,Gdor Mansilla - Comisaría,30mm,Tala


In [6]:
df["Localidad"].value_counts()

Localidad
Colón               120288
La Paz               25440
Paraná               21625
Gualeguaychú         21241
Nogoyá               19050
Feliciano            16152
Uruguay              15001
Villaguay            15000
Federacion           14496
Federal              13608
Tala                 12456
Concordia            10176
Gualeguay            10176
Victoria              7632
Islas del Ibicuy      5976
San Salvador          2544
Name: count, dtype: int64

In [ ]:
# 01-ELIMINAMOS ELEMENTOS NO DESEADOS ,a veces hay que usarloo
valores_a_eliminar = ["Dpto:", "Estación", "Dpto:Concordia", "Dpto:Diamante"]
# Filtrar el DataFrame para eliminar las filas que contienen los valores en la lista
df = df[~df["Estación"].isin(valores_a_eliminar)]

In [7]:
# 02- INCOORPORAMOS COORDENADAS DE LAS ESTACIONES
def asignar_coordenadas_estacion(estacion):
    coordenadas = {
        "Arroyo Barú": "31°51'57.7\"S 58°26'49.1\"W",
        "Colón - Obras Sanitarias": "32°13'01.1\"S 58°08'06.7\"W",
        "Mabragaña - Brigada Abigeato": "32°04'27.3\"S 58°15'11.3\"W",
        "Pueblo Cazes - Destacamento": "32°00'06.9\"S 58°30'01.2\"W",
        "Ubajay - Comisaría": "31°47'44.1\"S 58°18'56.6\"W",
        "Villa Elisa - Comisaría": "32°09'54.8\"S 58°24'04.1\"W",
        "Concordia- Ciudad": "31°23'02.6\"S 58°00'51.2\"W",
        "El Redomon - Destacamento": "31°06'02.4\"S 58°16'20.2\"W",
        "El Yuqueri - Destacamento": "31°23'01.2\"S 58°07'08.0\"W",
        "La Criolla - Comisaría": "31°16'11.7\"S 58°06'27.5\"W",
        "Los Charrúas - Comisaría": "31°10'06.9\"S 58°11'13.6\"W",
        "Nueva Escocia - Comisaría": "31°38'56.4\"S 58°00'57.0\"W",
        "Pedernal - Comisaría": "31°40'19.1\"S 58°13'46.7\"W",
        "Pto. Yeruá - Comisaría": "31°31'38.7\"S 58°00'53.3\"W",
        "Costa Grande - Comisaría": "32°08'56.4\"S 60°31'04.0\"W",
        "Diamante - Ciudad": "32°04'09.4\"S 60°39'06.6\"W",
        "Ejido Diamante - Comisaría": "32°06'47.3\"S 60°34'32.0\"W",
        "Isletas - Comisaría": "32°12'34.7\"S 60°19'55.1\"W",
        "Las Cuevas - Destacamento": "32°20'01.3\"S 60°29'09.6\"W",
        "Molino Doll - Destacamento": "32°18'21.9\"S 60°25'30.8\"W",
        "V. Libertador Gral. San Martín - Comisaría": "32°04'59.6\"S 60°29'09.9\"W",
        "Bella Unión - Destacamento": "30°37'09.7\"S 58°29'21.7\"W",
        "Chajarí - Ciudad": "30°45'21.6\"S 57°59'02.2\"W",
        "El Cerrito - Puesto Caminero": "30°38'49.1\"S 57°59'50.1\"W",
        "Federación - Corralón Municipal": "30°59'15.8\"S 57°56'06.2\"W",
        "Federación - Prefectura": "30°59'34.4\"S 57°54'53.8\"W",
        "Los Conquistadores": "30°35'26.6\"S 58°28'02.7\"W",
        "Mandisoví": "30°44'18.7\"S 58°01'21.2\"W",
        "San Jaime - Comisaría": "30°20'22.1\"S 58°18'23.1\"W",
        "San Jaime - Puesto Caminero": "30°16'58.0\"S 58°15'05.9\"W",
        "San Ramón - Comisaría Nº 20": "30°48'05.4\"S 58°11'16.3\"W",
        "Santa Ana - Comisaría": "30°54'05.2\"S 57°55'47.4\"W",
        "Sta. María de Tatutí - Comisaría Nº 22": "30°36'08.3\"S 58°10'37.2\"W",
        "Banderas": "30°47'21.6\"S 59°09'36.9\"W",
        "Chañar - Comisaría": "31°08'58.0\"S 58°42'20.0\"W",
        "Conscripto Bernardi - Comisaría": "31°02'54.7\"S 59°05'02.9\"W",
        "El Cimarrón - Comisaría": "30°58'24.4\"S 58°59'46.2\"W",
        "Elias Romero - Dto. Moreyra - Comisaría": "31°13'53.3\"S 58°35'26.0\"W",
        "Federal - Brigada Abigeato": "30°56'52.3\"S 58°47'52.7\"W",
        "Federal - Jefatura Departamental": "30°57'16.8\"S 58°46'26.0\"W",
        "Gral. Güemes - Destacamento": "31°18'17.1\"S 59°05'51.6\"W",
        "La Calandria - Comisaría": "30°45'48.6\"S 58°36'45.7\"W",
        "Nueva Vizcaya - Comisaría": "30°58'02.5\"S 58°36'52.9\"W",
        "Sauce de Luna - Comisaría": "31°14'21.7\"S 59°13'02.6\"W",
        "Atencio - Subcomisaría": "30°38'14.6\"S 58°38'13.2\"W",
        "Feliciano - Ciudad - Est.Agromet. DPH": "30°23'11.5\"S 58°45'13.7\"W",
        "La Esmeralda - Comis.Nº 15 (Basualdo)": "30°17'58.6\"S 58°39'56.9\"W",
    "La Hierra - Destacamento Nº 52": "30°27'08.6\"S 58°24'19.4\"W",
    "La Verbena - Comisaría": "30°23'37.1\"S 58°34'19.4\"W",
    "Las Mulitas - Comisaría Nº 19": "30°22'21.5\"S 58°58'50.1\"W",
    "Mulas Grandes - Destacamento Nº 54": "30°21'27.1\"S 59°08'20.5\"W",
    "Paso Bravo - Destacamento": "30°25'50.8\"S 58°45'17.5\"W",
    "Puente de Hierro - Destacamento Nº 50": "30°12'36.8\"S 58°47'14.5\"W",
    "Rincón de Mesa - Destacamento Nº 53": "30°18'05.2\"S 58°43'07.3\"W",
    "Rincón del Chañar - Destacamento Nº 55": "30°17'09.6\"S 59°04'11.7\"W",
    "San Pedro - Destacamento": "30°20'01.3\"S 58°20'39.0\"W",
    "San Víctor - Subcomisaría Nº 22": "30°29'14.2\"S 59°01'36.4\"W",
    "3º Distrito - Aldea Asunción": "32°49'28.9\"S 59°14'12.5\"W",
    "6º Distrito - Comisaría Nº 17 (3 Boc.)": "32°51'53.4\"S 59°42'15.1\"W",
    "7º Distrito - Comisaría Nº 28": "33°00'37.5\"S 59°29'15.6\"W",
    "8º Distrito - Comisaría Nº 27": "32°58'31.0\"S 59°29'34.3\"W",
    "Gral. Galarza - Comisaría": "32°43'09.2\"S 59°23'41.8\"W",
    "Gualeguay Ciudad - Bomberos Voluntarios": "33°08'14.9\"S 59°18'25.5\"W",
    "Lazo - Comisaría": "32°52'16.9\"S 59°25'21.2\"W",
    "Puerto Ruiz - Comisaría": "33°13'21.4\"S 59°21'49.4\"W",
    "Alarcón - Destacamento": "33°08'17.0\"S 59°04'03.2\"W",
    "Aldea San Antonio (Scheffer)": "32°37'31.2\"S 58°42'20.2\"W",
    "Carbó - Comisaría": "33°08'56.1\"S 59°12'08.8\"W",
    "Costa Río Uruguay - Comisaría": "33°11'25.5\"S 58°31'20.9\"W",
    "Costa San Antonio": "32°37'27.3\"S 59°04'04.9\"W",
    "Cuchilla Redonda - Comisaría": "33°05'10.5\"S 59°05'41.2\"W",
    "Escriña - Destacamento": "32°35'35.8\"S 58°54'15.4\"W",
    "Estación Gilbert - Comisaría": "32°31'57.1\"S 58°56'00.3\"W",
    "Faustino M. Parera - Destacamento": "32°48'08.4\"S 58°52'57.9\"W",
    "Gchú - Ciudad - Municipalidad": "33°00'10.1\"S 58°31'18.5\"W",
    "Gral. Almada - Comisaría": "32°50'15.2\"S 58°48'22.6\"W",
    "Irazusta - Subcomisaría": "32°55'42.3\"S 58°57'15.4\"W",
    "Pastor Britos": "32°45'26.4\"S 58°53'08.4\"W",
    "Pehuajó Sur - Subcomisaría": "33°03'57.1\"S 58°52'23.9\"W",
    "Perdices": "33°18'01.5\"S 58°42'40.4\"W",
    "Rincón del Gato - Comisaría": "32°52'06.1\"S 58°33'59.5\"W",
    "Talitas - Subcomisaría": "33°06'14.5\"S 59°12'22.1\"W",
    "Brazo Largo - Puesto Caminero": "33°46'25.9\"S 58°36'27.8\"W",
    "Holt Ibicuy - Comisaría": "33°44'44.7\"S 59°09'34.0\"W",
    "Mazaruca - Destacamento": "33°36'33.1\"S 59°16'41.3\"W",
    "Paranacito - Puesto Caminero": "33°35'06.4\"S 58°47'52.7\"W",
    "Villa Paranacito - Prefectura": "33°42'50.7\"S 58°39'41.8\"W",
    "Alcaraz Sur - Destacamento": "31°18'34.7\"S 59°49'57.3\"W",
    "Bovril - Comisaría": "31°20'27.6\"S 59°26'46.0\"W",
    "Colonia Avigdor": "31°11'01.3\"S 59°24'27.3\"W",
    "Colonia Oficial Nº 3": "30°33'27.0\"S 59°24'18.9\"W",
    "El Carmen - Yeso Oeste": "30°59'41.3\"S 59°26'51.6\"W",
    "El Quebracho - Destacamento Santa Elena": "30°59'08.2\"S 59°39'32.8\"W",
    "Estacas - Destacamento 53": "30°35'10.7\"S 59°15'38.5\"W",
    "Estaquitas - Comisaría Nº 27": "30°50'36.0\"S 59°37'39.0\"W",
    "La Lila - Particular": "31°01'53.5\"S 59°37'39.3\"W",
    "La Paz - Comisaría 1º": "30°44'27.5\"S 59°35'58.7\"W",
    "La Paz - Subprefectura": "30°44'17.4\"S 59°38'51.8\"W",
    "Las Toscas - Destacamento Nº 54": "30°29'37.3\"S 59°06'06.1\"W",
    "Paso Telegrafo - Puesto Caminero": "30°21'09.4\"S 59°30'47.9\"W",
    "Piedras Blancas": "31°11'02.1\"S 59°57'24.5\"W",
    "San Gustavo - Comisaría": "30°43'22.4\"S 59°28'11.2\"W",
    "Santa Elena - Prefectura": "30°56'39.6\"S 59°47'30.0\"W",
    "Tacuara Ombú - Comisaría Nº 24": "30°27'60.0\"S 59°14'32.6\"W",
    "Tacuara Yacaré - Comisaría Nº 23": "30°43'48.6\"S 59°30'33.2\"W",
    "Villa Alcaraz - Comisaría": "31°27'17.2\"S 59°35'52.4\"W",
    "Yeso Oeste -  Subcomisaría Nº 26": "30°58'39.2\"S 59°27'58.2\"W",
    "20 de Septiembre - Comisaría Nº 17": "32°23'34.2\"S 59°51'42.2\"W",
    "Algarrobito - Comisaría Nº 1": "32°16'19.9\"S 59°39'27.8\"W",
    "Aranguren - Comisaría": "32°14'41.4\"S 60°09'40.2\"W",
    "Betbeder - Comisaría": "32°22'21.1\"S 59°55'43.0\"W",
    "Crucesita 3ª - Comisaría": "32°07'28.1\"S 59°44'31.0\"W",
    "Crucesita 7ª - Escuela Nº 87": "32°02'25.7\"S 59°42'25.5\"W",
    "Crucesita 8ª - Comisaría Nº 27": "32°01'39.1\"S 59°42'00.4\"W",
    "Don Cristobal 2ª - Comisaría Nº 21": "32°04'19.8\"S 59°59'41.5\"W",
    "El Pueblito - Comisaría Nº 20": "32°09'50.7\"S 59°48'47.8\"W",
    "Febre - Comisaría Nº 19": "32°28'32.7\"S 59°55'09.5\"W",
    "Hernández - Comisaría": "32°20'10.5\"S 60°01'32.8\"W",
    "La Llave - Comisaría Nº 22": "32°31'58.4\"S 59°34'58.3\"W",
    "Lucas González - Est. Agromet. DPH": "32°23'04.6\"S 59°31'51.4\"W",
    "Nogoyá - Puesto Caminero": "32°22'59.9\"S 59°47'07.7\"W",
    "Sección Urquiza - Comisaría Nº 28": "31°58'11.1\"S 59°42'39.2\"W",
        "Arroyo María - Nidia Faez": "31°46'34.4\"S 59°52'10.7\"W",
        "Cerrito - Comisaría": "31°34'46.0\"S 60°04'29.6\"W",
        "Curtiembre - Destacamento": "31°27'32.6\"S 60°10'10.0\"W",
        "El Espinillo": "31°48'40.6\"S 60°16'17.1\"W",
        "Engenfeld": "31°53'09.9\"S 60°14'05.9\"W",
        "Hasenkamp - Comisaría": "31°30'41.4\"S 59°50'09.7\"W",
        "Hernandarias - Comisaría": "31°13'48.8\"S 59°59'04.7\"W",
        "La Picada": "31°44'05.2\"S 60°18'41.5\"W",
        "Paraná - Escuela de Policía": "31°43'45.5\"S 60°29'40.5\"W",
        "Pueblo Bellocq - Destacamento Las Garzas": "31°25'41.5\"S 59°44'35.8\"W",
        "Pueblo Brugo - Comisaría": "31°23'08.8\"S 60°05'46.0\"W",
        "Quebracho - El Ramblón": "31°50'36.3\"S 60°05'18.9\"W",
        "Seguí - Comisaría": "31°57'19.7\"S 60°07'30.1\"W",
        "Tabossi - Municipalidad": "31°48'09.9\"S 59°56'18.0\"W",
        "Viale - Comisaría": "31°52'15.4\"S 60°00'32.4\"W",
        "Villa Fontana - Comisaría": "31°51'43.0\"S 60°30'17.9\"W",
        "Villa Urquiza - Comisaría": "31°38'39.8\"S 60°22'39.1\"W",
        "General Campos - Comisaría": "31°31'28.0\"S 58°24'25.8\"W",
        "San Salvador - Comisaría": "31°37'39.6\"S 58°30'33.6\"W",
        "Altamirano Sur": "32°05'22.6\"S 59°10'27.7\"W",
        "Arroyo Clé - Comisaría": "32°38'18.4\"S 59°23'55.1\"W",
        "Durazno - Comisaría": "31°59'21.9\"S 59°16'45.8\"W",
        "Gdor Echagüe - Comisaría": "32°23'34.1\"S 59°16'35.2\"W",
        "Gdor Mansilla - Comisaría": "32°32'43.1\"S 59°21'18.8\"W",
        "Gdor. Maciá - Comisaría": "32°10'23.5\"S 59°24'02.2\"W",
        "Guardamonte - Comisaría": "32°04'52.0\"S 59°18'18.2\"W",
        "Rosario del Tala - OSM": "32°18'28.7\"S 59°08'38.2\"W",
        "Sauce Norte": "32°25'14.4\"S 59°14'16.4\"W",
        "Sauce Sur": "32°36'27.1\"S 59°12'15.2\"W",
        "Tala - Ciudad": "32°18'21.3\"S 59°08'39.8\"W",
        "1° de Mayo - Comisaría": "32°15'40.7\"S 58°25'15.6\"W",
        "Basavilbaso - Comisaría": "32°22'45.0\"S 58°52'22.1\"W",
        "Colonia Elias - Comisaría": "32°40'15.4\"S 58°19'33.1\"W",
        "Estación Herrera - Comisaría": "32°26'05.5\"S 58°37'44.2\"W",
        "Las Moscas - Subcomisaría": "32°05'30.8\"S 58°57'19.5\"W",
        "Libaros - Subcomisaría": "32°15'40.2\"S 58°54'24.1\"W",
        "Pronunciamiento - Comisaría": "32°20'51.0\"S 58°26'33.9\"W",
        "Rocamora - Comisaría": "32°20'26.1\"S 58°58'06.5\"W",
        "Santa Anita - Comisaría": "32°10'28.7\"S 58°47'00.7\"W",
        "Sauce - Comisaría": "32°36'57.8\"S 58°27'08.2\"W",
        "Talita - Comisaría": "32°32'03.8\"S 58°20'19.8\"W",
        "Uruguay - Ciudad": "32°20'34.3\"S 58°09'44.5\"W",
        "Villa Mantero - Comisaría": "32°23'56.1\"S 58°44'43.2\"W",
        "Antelo - Sara de Muzzio": "32°31'56.7\"S 60°02'12.8\"W",
        "Hinojal - Comisaría Nº 23": "32°22'44.2\"S 60°10'44.4\"W",
        "Laguna del Pescado - Brigada Abigeato": "32°41'12.2\"S 60°00'42.9\"W",
        "Montoya - Comisaría Nº 20": "32°36'15.2\"S 59°52'10.3\"W",
        "Pajonal": "32°26'19.8\"S 60°18'00.9\"W",
        "Rincón del Nogoyá - Comisaría Nº 17": "32°46'44.7\"S 59°54'37.0\"W",
        "Brigada Abigeato": "33°36'20.4\"S 58°46'41.0\"W",
        "Comisaría 1°": "31°50'46.9\"S 59°01'10.7\"W",
        "Ingeniero Sajaroff - Subcomisaría": "31°57'23.5\"S 58°50'55.1\"W",
        "Jubileo - Comisaría": "31°43'59.0\"S 58°38'01.4\"W",
        "Lucas Norte - Subcomisaría": "31°28'45.3\"S 58°49'45.8\"W",
        "Lucas Sur I - Subcomisaría": "31°38'44.4\"S 59°00'52.7\"W",
        "Mojones Norte - Comisaría Nº 22": "31°26'42.5\"S 59°11'10.2\"W",
        "Mojones Sur I - Comisaría Nº 27": "31°37'23.1\"S 59°10'55.6\"W",
        "Paso de La Laguna - Raíces Oeste": "31°48'08.4\"S 59°09'34.0\"W",
        "Raíces Oeste - Comisaría Nº 29": "31°54'32.7\"S 59°15'41.7\"W",
        "Villa Clara - Comisaría": "31°49'38.8\"S 58°49'19.3\"W",
        "Villa Dominguez - Comisaría": "31°59'13.8\"S 58°57'54.5\"W",
        "Zenon Roca - Comisaría": "31°33'22.4\"S 58°43'49.6\"W",

    }
    return coordenadas.get(estacion, None)  # Retorna None si la estación no está en el diccionario

# Aplicar la función a la columna 'Estación' y almacenar el resultado en una nueva columna 'Coordenadas_Estacion'
df['Coordenadas_Estacion'] = df['Estación'].apply(asignar_coordenadas_estacion)


In [8]:
# 03- ELIMINAMOS FILAS NULAS QUE QUEDARON A LA DERIVA
df = df.dropna()

In [9]:
# 04- ELIMINAMOS LOS "mm" DE LA COLUMNMA Precipitaciones
def remove_mm(value):
    return re.sub(r'mm$', '', value)
# Aplica la función a la columna "Precipitación"
df['Precipitación'] = df['Precipitación'].apply(remove_mm)

<ipython-input-9-d32a107288ac>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Precipitación'] = df['Precipitación'].apply(remove_mm)


In [10]:
# 05- TRANSFORMAMOS '---' A NaN EN LA COLUMNA 'Precipitación'
df['Precipitación'] = df['Precipitación'].apply(lambda x: pd.NA if x == '---' else x)

<ipython-input-10-9b11bdd6bbb3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Precipitación'] = df['Precipitación'].apply(lambda x: pd.NA if x == '---' else x)


In [11]:
# 06- TRANSFORMAMOS LOS NaN A NUMEROS PARA  LUEGO PODER PASARLO A FLOAT
# Reemplazar los valores NaN con 123456789
df['Precipitación'].fillna(123456789, inplace=True)

# Convertir la columna "Precipitación" a tipo float
df['Precipitación'] = df['Precipitación'].astype(float)

# Reemplazar los valores 123456789 con NaN
df['Precipitación'].replace(123456789, np.nan, inplace=True)

<ipython-input-11-fa56afb558cf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Precipitación'].fillna(123456789, inplace=True)
<ipython-input-11-fa56afb558cf>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Precipitación'] = df['Precipitación'].astype(float)
<ipython-input-11-fa56afb558cf>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Precipitación'].replace(123

In [12]:
# 07- TRANSFORMAMOS LA COLUMNA 'fecha' a datetime
df['fecha'] = pd.to_datetime(df['fecha'])

<ipython-input-12-4376ad2e089d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fecha'] = pd.to_datetime(df['fecha'])


In [13]:
# 08- EXTRAEMOS LATITUD Y LONG.
df[['Latitud', 'Longitud']] = df['Coordenadas_Estacion'].str.extract(r'(\d+°\d+\'\d+\.\d+"\w)\s(\d+°\d+\'\d+\.\d+"\w)')

# Función para convertir coordenadas en formato grados, minutos y segundos a decimal
def dms_to_dd(dms_str):
    dms_str = dms_str.replace("°", " ").replace("'", " ").replace('"', ' ').replace("’", " ")
    parts = re.split(r'\s+', dms_str)
    degrees = float(parts[0])
    minutes = float(parts[1])
    seconds = float(parts[2].strip('S|N|E|W'))
    direction = parts[3].strip()

    dd = degrees + (minutes / 60.0) + (seconds / 3600.0)

    if direction in ['S', 'W']:
        dd *= -1

    return dd

# Convertir las coordenadas
df['Latitud_dd'] = df['Latitud'].apply(dms_to_dd)
df['Longitud_dd'] = df['Longitud'].apply(dms_to_dd)

<ipython-input-13-0da5c6ad8cda>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['Latitud', 'Longitud']] = df['Coordenadas_Estacion'].str.extract(r'(\d+°\d+\'\d+\.\d+"\w)\s(\d+°\d+\'\d+\.\d+"\w)')
<ipython-input-13-0da5c6ad8cda>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['Latitud', 'Longitud']] = df['Coordenadas_Estacion'].str.extract(r'(\d+°\d+\'\d+\.\d+"\w)\s(\d+°\d+\'\d+\.\d+"\w)')
<ipython-input-13-0da5c6ad8cda>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

In [14]:
# 09- ELIMINACION DE COLUMNAS IRRELEVANTES
df = df.drop(columns=['Latitud'])
df = df.drop(columns=['Longitud'])

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 239562 entries, 0 to 330860
Data columns (total 7 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   fecha                 239562 non-null  datetime64[ns]
 1   Estación              239562 non-null  object        
 2   Precipitación         143681 non-null  float64       
 3   Localidad             239562 non-null  object        
 4   Coordenadas_Estacion  239562 non-null  object        
 5   Latitud_dd            239562 non-null  float64       
 6   Longitud_dd           239562 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 14.6+ MB


In [16]:
# Asumiendo que ya tienes tu DataFrame cargado en la variable df
df = df.drop(columns=['Coordenadas_Estacion'])

In [17]:
df.shape

(239562, 6)

In [18]:
df

,fecha,Estación,Precipitación,Localidad,Latitud_dd,Longitud_dd
0,2010-09-02,Durazno - Comisaría,4.0,Tala,-31.989417,-59.279389
1,2010-09-02,Gdor Echagüe - Comisaría,5.0,Tala,-32.392806,-59.276444
2,2010-09-02,Gdor Mansilla - Comisaría,5.0,Tala,-32.545306,-59.355222
3,2010-09-02,Gdor. Maciá - Comisaría,11.0,Tala,-32.173194,-59.400611
4,2010-09-02,Rosario del Tala - OSM,8.0,Tala,-32.307972,-59.143944
...,...,...,...,...,...,...
330856,2024-05-22,Brazo Largo - Puesto Caminero,0.0,Islas del Ibicuy,-33.773861,-58.607722
330857,2024-05-22,Holt Ibicuy - Comisaría,0.0,Islas del Ibicuy,-33.745750,-59.159444
330858,2024-05-22,Mazaruca - Destacamento,0.0,Islas del Ibicuy,-33.609194,-59.278139
330859,2024-05-22,Paranacito - Puesto Caminero,0.0,Islas del Ibicuy,-33.585111,-58.797972


In [19]:
# Guarda el DataFrame como un archivo CSV
df.to_csv('df_limpio.csv', index=False)